In [1]:
!pip install gradio pdf2image pytesseract nltk matplotlib PyMuPDF
!apt-get install -y poppler-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 45.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (233 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
#Import Libraries and Setup

import gradio as gr
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import re
import tempfile
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from pdf2image import convert_from_bytes
import nltk

nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#Define Skills for Roles

RECOMMENDED_SKILLS = {
    "AI/ML Intern": [
        "python", "tensorflow", "pytorch", "pandas", "numpy",
        "deep learning", "machine learning", "scikit-learn", "matplotlib", "nlp"
    ],
    "Data Analyst": [
        "excel", "sql", "power bi", "tableau", "python",
        "data visualization", "data cleaning", "statistics", "pandas", "numpy"
    ],
    "Web Developer": [
        "html", "css", "javascript", "react", "next.js",
        "flask", "node.js", "bootstrap", "api integration", "mongodb"
    ],
    "Backend Developer": [
        "python", "django", "flask", "rest api", "fastapi",
        "sql", "postgresql", "mongodb", "docker", "authentication"
    ],
    "Cloud Engineer": [
        "aws", "azure", "gcp", "docker", "kubernetes",
        "terraform", "cloudwatch", "lambda", "devops", "ci/cd"
    ],
    "DevOps Engineer": [
        "jenkins", "git", "docker", "kubernetes", "linux",
        "terraform", "ansible", "ci/cd", "aws", "monitoring"
    ],
    "Cybersecurity Analyst": [
        "network security", "vulnerability assessment", "siem", "wireshark", "firewalls",
        "ethical hacking", "nmap", "linux", "burpsuite", "incident response"
    ],
    "Data Scientist": [
        "python", "r", "sql", "pandas", "numpy",
        "tensorflow", "pytorch", "scikit-learn", "mlops", "data storytelling"
    ],
    "Business Analyst": [
        "excel", "sql", "power bi", "tableau", "communication",
        "requirement gathering", "data analysis", "stakeholder management", "agile"
    ],
    "Full Stack Developer": [
        "html", "css", "javascript", "react", "node.js",
        "mongodb", "express.js", "python", "sql", "api"
    ]
}


In [4]:
#Functions for Resume Analysis

# Extract text from uploaded resume
def extract_text_from_resume(resume_bytes):
    try:
        images = convert_from_bytes(resume_bytes)
        text = "\n".join([pytesseract.image_to_string(img) for img in images])
        return text
    except Exception as e:
        return f"[ERROR] Unable to extract text: {str(e)}"

# Get keywords from job description
def extract_keywords_from_jd(jd_text):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(jd_text.lower())
    stop_words = set(stopwords.words("english"))
    keywords = [word for word in words if word not in stop_words]
    return list(set(keywords))

# Get skills from resume
def extract_skills(resume_text):
    all_skills = set()
    for skill_list in RECOMMENDED_SKILLS.values():
        all_skills.update(skill_list)
    found = [skill for skill in all_skills if skill.lower() in resume_text.lower()]
    return found

# Match roles based on found skills
def match_roles(skills_found):
    matched_roles = []
    total_skills = set().union(*RECOMMENDED_SKILLS.values())

    for role, required in RECOMMENDED_SKILLS.items():
        match = list(set(skills_found) & set(required))
        if match:
            score = int((len(match) / len(required)) * 100)
            matched_roles.append({
                "Role": role,
                "Matched": match,
                "Score": score
            })
    ats_score = int((len(skills_found) / len(total_skills)) * 100)
    return matched_roles, ats_score


In [5]:
#Create Pie Chart + Smart Message

def plot_ats_score(score):
    fig, ax = plt.subplots()
    labels = [f"Match ({score}%)", f"Missing ({100 - score}%)"]
    colors = ["#00C9A7", "#F67280"]
    ax.pie([score, 100 - score], labels=labels, colors=colors, startangle=90, autopct='%1.1f%%')
    ax.axis("equal")
    plt.title("ATS Match Score", fontsize=14)
    temp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    plt.savefig(temp.name, bbox_inches="tight")
    plt.close(fig)
    return temp.name

def analyze_resume(resume_file, jd_text):
    resume_text = extract_text_from_resume(resume_file)
    if resume_text.startswith("[ERROR]"):
        return None, resume_text

    jd_keywords = extract_keywords_from_jd(jd_text)
    resume_skills = extract_skills(resume_text)
    matched_roles, ats_score = match_roles(resume_skills)
    pie_path = plot_ats_score(ats_score)

    if ats_score < 60:
        message = f"😕 Your ATS score is {ats_score}%, which is a bit low.\n\n"
        message += "🔍 To improve your chances, try refining your resume to highlight more relevant skills."
    else:
        best = sorted(matched_roles, key=lambda x: x['Score'], reverse=True)[0]
        message = f"✅ Your ATS score is {ats_score}% — you're a strong match!\n\n"
        message += f"💼 Best fit role: {best['Role']} ({best['Score']}%)\n"
        others = [r for r in matched_roles if r['Role'] != best['Role']]
        if others:
            message += "\n📌 You might also explore:\n"
            for r in others[:2]:
                message += f"- {r['Role']} ({r['Score']}%)\n"

    return pie_path, message



In [6]:
#Build Gradio Interface

demo = gr.Interface(
    fn=analyze_resume,
    inputs=[
        gr.File(type="binary", label="📄 Upload Resume"),
        gr.Textbox(label="📝 Paste Job Description", placeholder="Paste JD here...", lines=8)
    ],
    outputs=[
        gr.Image(type="filepath", label="🎯 ATS Score"),
        gr.Markdown(label="💡 Suggestions & Best Fit Role")
    ],
    title="🚀 Resume Analyzer based on JD",
    description="Upload your resume and paste the job description to get your ATS match score and recommendations!",
    theme="default"
)

demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25965bfedcfdf17d64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
